In [75]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [76]:
# Import our 3-year modeling data
data = pd.read_csv('data/all_model_data.csv')
data

,Age,City_Eugene,City_Springfield,DayOfMonth,DayOfWeek,DayOfYear,Gender_female,Gender_male,Gender_non_binary,Gender_trans_female,...,sunset_hour,temp,tempmax,tempmin,uvindex,visibility,windgust,windspeed,Date,CallsPerDay
0,34,1,0,10,5,253,0,1,0,0,...,19,70,89,51,6,8,15,12,1662768000000000000,1
1,38,1,0,1,4,244,0,1,0,0,...,19,67,78,62,4,7,19,15,1693526400000000000,3
2,48,0,1,1,4,244,0,0,1,0,...,19,67,78,62,4,7,19,15,1693526400000000000,3
3,11,0,1,1,4,244,0,1,0,0,...,19,67,78,62,4,7,19,15,1693526400000000000,3
4,26,1,0,2,5,245,1,0,0,0,...,19,67,85,55,8,9,15,11,1693612800000000000,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,65,1,0,14,5,287,0,1,0,0,...,18,56,64,48,4,7,9,4,1697241600000000000,3
345,70,0,1,15,6,288,1,0,0,0,...,18,59,69,54,6,6,5,6,1697328000000000000,4
346,32,1,0,15,6,288,1,0,0,0,...,18,59,69,54,6,6,5,6,1697328000000000000,4
347,39,1,0,15,6,288,1,0,0,0,...,18,59,69,54,6,6,5,6,1697328000000000000,4


In [77]:
# Import our upcoming prediction data
upcoming_data = pd.read_csv('data/upcoming_data_with_dummies.csv')
upcoming_data

,DayOfMonth,DayOfWeek,DayOfYear,Month,Season_Autumn,Season_Spring,Season_Summer,Season_Winter,Year,cloudcover,...,solarradiation,sunrise_hour,sunset_hour,temp,tempmax,tempmin,uvindex,visibility,windgust,windspeed
0,14,1,135,5,False,True,False,False,2024,12.8,...,323.4,5,20,60.9,74.1,47.9,9,10.0,26.4,18.2
1,15,2,136,5,False,True,False,False,2024,10.2,...,330.0,5,20,63.7,78.0,49.0,9,10.1,17.2,11.4
2,16,3,137,5,False,True,False,False,2024,20.5,...,344.4,5,20,62.2,74.0,51.0,9,10.1,13.9,9.2
3,17,4,138,5,False,True,False,False,2024,26.2,...,344.4,5,20,57.0,65.9,49.0,9,11.3,18.3,12.8
4,18,5,139,5,False,True,False,False,2024,10.5,...,327.0,5,20,55.2,66.8,43.6,9,15.0,19.0,12.3
5,19,6,140,5,False,True,False,False,2024,28.8,...,274.4,5,20,53.5,63.6,44.0,8,14.9,18.1,12.1
6,20,0,141,5,False,True,False,False,2024,24.5,...,346.9,5,20,53.3,63.4,43.2,9,15.0,17.0,11.4
7,21,1,142,5,False,True,False,False,2024,34.3,...,327.2,5,20,54.4,64.5,44.3,9,15.0,16.8,11.0


In [78]:
age_column = ['Age']
gender_columns = ['Gender_female', 'Gender_male', 'Gender_non_binary', 'Gender_trans_female', 'Gender_trans_male']
race_columns = ['Race_alaska native', 'Race_american indian', 'Race_american indian/alaska native', 'Race_asian',
                'Race_black/african american', 'Race_hispanic/latino', 'Race_native hawaiian/other pacific islander',
                'Race_other', 'Race_two or more races', 'Race_white']
city_columns = ['City_Eugene', 'City_Springfield']
reason_for_dispatch_columns = ['Reason for Dispatch_Check Welfare', 'Reason for Dispatch_Counseling',
                                'Reason for Dispatch_EMS Assist', 'Reason for Dispatch_Fire Assist',
                                'Reason for Dispatch_Police Assist', 'Reason for Dispatch_Public Assist',
                                'Reason for Dispatch_Suicidal Subject', 'Reason for Dispatch_Transport']

classification_columns = age_column + gender_columns + race_columns + city_columns + reason_for_dispatch_columns

model_data = data.drop(columns=classification_columns)
print(model_data.columns)
model_data.head()

Index(['DayOfMonth', 'DayOfWeek', 'DayOfYear', 'Hour', 'Month',
       'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
       'Year', 'cloudcover', 'conditions_Clear', 'conditions_Overcast',
       'conditions_Partially cloudy', 'conditions_Rain',
       'conditions_Rain, Freezing Drizzle/Freezing Rain, Overcast',
       'conditions_Rain, Freezing Drizzle/Freezing Rain, Partially cloudy',
       'conditions_Rain, Overcast', 'conditions_Rain, Partially cloudy',
       'conditions_Snow, Rain', 'conditions_Snow, Rain, Overcast',
       'conditions_Snow, Rain, Partially cloudy', 'date', 'dew', 'feelslike',
       'feelslikemax', 'feelslikemin', 'humidity', 'pm25', 'precip',
       'precipcover', 'precipprob', 'sealevelpressure', 'snow', 'snowdepth',
       'solarenergy', 'solarradiation', 'sunrise_hour', 'sunset_hour', 'temp',
       'tempmax', 'tempmin', 'uvindex', 'visibility', 'windgust', 'windspeed',
       'Date', 'CallsPerDay'],
      dtype='object')


,DayOfMonth,DayOfWeek,DayOfYear,Hour,Month,Season_Autumn,Season_Spring,Season_Summer,Season_Winter,Year,...,sunset_hour,temp,tempmax,tempmin,uvindex,visibility,windgust,windspeed,Date,CallsPerDay
0,10,5,253,11,9,1,0,0,0,2022,...,19,70,89,51,6,8,15,12,1662768000000000000,1
1,1,4,244,12,9,1,0,0,0,2023,...,19,67,78,62,4,7,19,15,1693526400000000000,3
2,1,4,244,17,9,1,0,0,0,2023,...,19,67,78,62,4,7,19,15,1693526400000000000,3
3,1,4,244,20,9,1,0,0,0,2023,...,19,67,78,62,4,7,19,15,1693526400000000000,3
4,2,5,245,12,9,1,0,0,0,2023,...,19,67,85,55,8,9,15,11,1693612800000000000,7


#### Finding Weights with OLS

In [79]:
# Split the data into features (X) and target variable (y)
X = model_data.drop(columns=['CallsPerDay'])
y = model_data['CallsPerDay']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train an OLS regression model
reg = LinearRegression()
reg.fit(X_train, y_train)

# Predict on the test set
y_pred = reg.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Print the weights (coefficients) of each feature, sorted by absolute value
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': reg.coef_
})

coefficients['AbsCoefficient'] = coefficients['Coefficient'].abs()
coefficients_sorted = coefficients.sort_values(by='AbsCoefficient', ascending=False).drop(columns='AbsCoefficient')

print("Coefficients sorted by absolute value:")
print(coefficients_sorted)

# Save the sorted coefficients to a CSV file
coefficients_sorted.to_csv('data/OLS_coefficients.csv', index=False)

# Check for data leakage
X_leakage = X_test[X_test.index.isin(X_train.index)]
if not X_leakage.empty:
    print("Warning: Data Leakage Detected!\n")

Mean Squared Error: 12.00691071175353
R-squared: 0.4572571979262895
Coefficients sorted by absolute value:
                                              Feature   Coefficient
42                                            uvindex  1.662936e+00
23                                                dew  4.488326e-01
1                                           DayOfWeek -2.555629e-01
43                                         visibility  2.397390e-01
2                                           DayOfYear  2.308716e-01
45                                          windspeed -2.145240e-01
38                                        sunset_hour  2.027608e-01
26                                       feelslikemin -1.993456e-01
32                                   sealevelpressure  1.887130e-01
35                                        solarenergy  1.840629e-01
25                                       feelslikemax  1.567368e-01
44                                           windgust  1.388444e-01
28       

#### Risk Classification

In [80]:
# Convert the column names of both DataFrames to sets
model_data_columns = set(model_data.columns)
upcoming_data_columns = set(upcoming_data.columns)

# Columns in model_data but not in upcoming_data
model_only_columns = model_data_columns.difference(upcoming_data_columns)
print("Columns in model_data but not in upcoming_data:")
print(model_only_columns)

# Columns in upcoming_data but not in model_data
upcoming_only_columns = upcoming_data_columns.difference(model_data_columns)
print("\nColumns in upcoming_data but not in model_data:")
print(upcoming_only_columns)

# Drop columns in model_data but not in upcoming_data
columns_to_drop = ['date', 'Date', 'Hour']
model_data = model_data.drop(columns=columns_to_drop)

# List of columns to remove from both DataFrames
columns_to_drop = ['Year']
# Drop specified columns from model_data
model_data = model_data.drop(columns=columns_to_drop)
# Drop specified columns from upcoming_data
upcoming_data = upcoming_data.drop(columns=columns_to_drop)

Columns in model_data but not in upcoming_data:
{'Date', 'date', 'Hour', 'CallsPerDay'}

Columns in upcoming_data but not in model_data:
set()


In [81]:
# Calculate the call volume threshold for the 90th percentile
call_volume_threshold = model_data['CallsPerDay'].quantile(0.9)

# Label the days based on whether their call volume exceeds the threshold
model_data['HighRisk'] = (model_data['CallsPerDay'] >= call_volume_threshold).astype(int)

# Split the data into features (X) and target labels (y)
X = model_data.drop(columns=['CallsPerDay', 'HighRisk'])
y = model_data['HighRisk']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a Random Forest classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Check label distribution
label_distribution = model_data['HighRisk'].value_counts(normalize=True)
print("Label Distribution:")
print(label_distribution)

# Check for data leakage
X_leakage = X_test[X_test.index.isin(X_train.index)]
if not X_leakage.empty:
    print("Warning: Data Leakage Detected!\n")

Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        64
           1       1.00      1.00      1.00         6

    accuracy                           1.00        70
   macro avg       1.00      1.00      1.00        70
weighted avg       1.00      1.00      1.00        70

Label Distribution:
HighRisk
0    0.888252
1    0.111748
Name: proportion, dtype: float64


In [82]:
# Use the trained classifier to predict on upcoming_data
upcoming_data_predictions = clf.predict(upcoming_data)

# Add the predictions as a new column to upcoming_data
upcoming_data['HighRisk'] = upcoming_data_predictions

# Print the mean of the 'HighRisk' column in upcoming_data
print(upcoming_data['HighRisk'].mean())

0.0


In [83]:
# Save the DataFrame with predictions to a CSV file
upcoming_data.to_csv('data/upcoming_data_with_predictions.csv', index=False)

In [84]:
# Get feature importances
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': clf.feature_importances_})
feature_importances.sort_values(by='Importance', ascending=False, inplace=True)
print(feature_importances)

                                              Feature  Importance
2                                           DayOfYear    0.118696
25                                               pm25    0.117983
24                                           humidity    0.084100
20                                                dew    0.071344
33                                     solarradiation    0.058285
1                                           DayOfWeek    0.049529
41                                           windgust    0.048673
29                                   sealevelpressure    0.047256
39                                            uvindex    0.045196
32                                        solarenergy    0.043240
0                                          DayOfMonth    0.036934
3                                               Month    0.032689
22                                       feelslikemax    0.031275
36                                               temp    0.030616
8         